<a href="https://colab.research.google.com/github/AdrianaGRO/duty-free-sales/blob/main/DutyFreeSakesAnalyzer_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Py.3 - Duty-Free Sales Analyzer.ipynb**

In [1]:
!pip install openai


In [11]:
# Step 1: Import pandas
import pandas as pd
import openai
from google.colab import userdata
import os

#Step 2 : Set env

openai.api_key = userdata.get('OPEN_AI')
if not openai.api_key:
  raise ValueError("OPENAI API key not found in Colabs Secrets as 'OPEN_AI'.")

#Step 3: Load the Excel
df = pd.read_excel("/duty_free_sales.xlsx")

#Step 4: Basic analysis
category_summary = df.groupby("Category")[["Revenue (RON)", "Margin (RON)"]].sum()
top_margin_products = df.groupby("Product")["Margin (RON)"].sum().nlargest(3)

#Step 5: Define a simple AI agent:
def sales_analyzer_agent(dataframe):
  total_revenue = dataframe["Revenue (RON)"].sum()
  total_margin = dataframe["Margin (RON)"].sum()
  top_revenue_products = dataframe.groupby("Product")["Revenue (RON)"].sum().nlargest(3)
  top_margin_products = dataframe.groupby("Product")["Margin (RON)"].sum().nlargest(3)
  category_data = dataframe.groupby("Category")[["Revenue (RON)", "Margin (RON)"]].sum()


  # Create a prompt for OpenAI
  prompt = (
      f"Travel Retail Romania - Duty free sales report:\n:"
      f"Total Revenue: {total_revenue:.2f} RON\n"
      f"Total Margin: {total_margin:.2f} RON\n"
      f"Top 3 Products by Revenue:\n{top_revenue_products}\n"
      f"Top 3 Products by Margin:\n{top_margin_products}\n"
      f"Category Summary\n{category_data}\n"
      f"Provide a concise, actionabla summary for duty-free managers, focusing on the insights and recomandatios(avoid repeting numbers)"
  )

  response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful sales analyst."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
  #Extract the summary
  summary = response.choices[0].message.content.strip()

  #save to a text file
  with open("sales_summary.txt", "w") as f:
    f.write(summary)
  return summary


# Step 6: Display result

print(" Travel Retail Romania - Duty Free Sales Data Updated ( April. 2025)")
print("-----------------------------------------------------------------------")
print("First 5 rows of data:")
print(df.head())
print("\nRevenue and margin by Category:")
print(category_summary)
print("\nTop 3 products by margin:")
print(top_margin_products)
print(f"Total Revenue: {df['Revenue (RON)'].sum():.2f} RON. This is total gross.")
print(f"Total Margin: {df['Margin (RON)'].sum():.2f} RON. This is total net.")
print(f"Total Units Sold: {df['Units Sold'].sum()}.")
print("-----------------------------------------------------------------------")
print("\nAI Agent Summary:")
agent_summary = sales_analyzer_agent(df)
print(agent_summary)
print("-----------------------------------------------------------------------")




 Travel Retail Romania - Duty Free Sales Data Updated ( April. 2025)
-----------------------------------------------------------------------
First 5 rows of data:
         Date           Product    Category  Price (RON)  \
0  2025-03-01  Ray-Ban Wayfarer  Sunglasses       358.74   
1  2025-03-01        Pepsi 0.5l   Beverages        11.72   
2  2025-03-01      Baileys 0.7l     Spirits       181.99   
3  2025-03-01        Pepsi 0.5l   Beverages        13.81   
4  2025-03-01      Dom Perignon     Spirits       113.61   

   Acquisition Price (RON)  Units Sold  Revenue (RON)  Margin (RON)  
0                   231.00          44       15784.56       5620.56  
1                     7.50           3          35.16         12.66  
2                   126.91          47        8553.53       2588.76  
3                     8.26          17         234.77         94.35  
4                    80.64          15        1704.15        494.55  

Revenue and margin by Category:
             Revenue (R